In [ ]:
import numpy as np
import datetime
import pandas as pd
import matplotlib.pyplot as plt
import dufte

plt.style.use(dufte.style)


def weighted_dates2series(
    participants: np.ndarray,
    dates: np.ndarray,
) -> pd.Series:
    start = min(dates)
    end = max(dates)
    index = [start + i * datetime.timedelta(days=1) for i in range((end - start).days)]
    # terribly inefficient
    values = [participants[dates < ind].sum() for ind in index]
    return pd.Series(values, index=index)

In [ ]:
from pathlib import Path

data_dir = Path(".").resolve().parent / "data"
df = pd.read_csv(data_dir / "event-data - Sheet1.csv")
df["Start date"] = pd.to_datetime(df["Start date"])

In [ ]:
date2parts = weighted_dates2series(df["Registrations"], df["Start date"])
teach_only = df.query("type == 'teach'")
date2parts_teach = weighted_dates2series(
    teach_only["Registrations"], teach_only["Start date"]
)

In [ ]:
ax = date2parts.plot(label="All")
date2parts_teach.plot(ax=ax, label="Teaching only", ls=":")
ax.legend(frameon=False)
ax.set_ylabel("Registrations")
ax.set_xlabel("Event start date")

**Still excluding outreach**
